Problem 1

This is essentially the same as problem 4 in assignment 1

Let’s say we have an exam question which consists of 20 yes/no questions. From past performance of
similar students, a randomly chosen student will know the correct answer to 𝑁 ∼ binom(20, 11/20)
questions. Furthermore, we assume that the student will guess the answer with equal probability
to each question they don’t know the answer to, i.e. given 𝑁 we define 𝑍 ∼ binom(20 − 𝑁 , 1/2) as
the number of correctly guessed answers. Define 𝑌 = 𝑁 + 𝑍, i.e., 𝑌 represents the number of total
correct answers.
We are interested in setting a deterministic threshold 𝑇 , i.e., we would pass a student at threshold
𝑇 if 𝑌 ≥ 𝑇 . Here 𝑇 ∈ {0, 1, 2, ... , 20}.
1. [5p] For each threshold 𝑇 , compute the probability that the student knows less than
10 correct answers given that the student passed, i.e., 𝑁 < 10. Put the answer in
problem11_probabilities as a list.
2. [3p] What is the smallest value of 𝑇 such that if 𝑌 ≥ 𝑇 then we are 90% certain that 𝑁 ≥ 10?

In [50]:
# We want to find P(N<10|Y>=T) (A|B)

In [1]:
from scipy.special import binom as binomial
from scipy.stats import binom as binomial_pmf
from sklearn.linear_model import LogisticRegression

n_questions = 20
p_know = 11 / 20
p_guess = 1 / 2
p_N = lambda k: binomial(n_questions, k) * ((1 - p_know) ** (n_questions - k)) * (
            (p_know) ** k)  # the PMF of N is p_N(k) where p_N is


def prob_Y_given_N(y, k):
    return binomial_pmf.pmf(y - k, n_questions - k, p_guess)


def prob_Y(y):
    return sum(p_N(k) * prob_Y_given_N(y, k) for k in range(n_questions + 1))


def prob_Y_geq_T(T):
    return sum(prob_Y(y) for y in range(T, n_questions + 1))


def prob_N_given_Y(T):
    prob = 0
    for k in range(5):
        for y in range(T, n_questions + 1):
            prob += p_N(k) * prob_Y_given_N(y, k)
    return prob


problem11_probabilities = []

for T in range(n_questions + 1):
    p_N_given_Y = prob_N_given_Y(T)
    p_Y_geq_T = prob_Y_geq_T(T)
    if p_Y_geq_T > 0:
        cond_prob = p_N_given_Y / p_Y_geq_T
    else:
        cond_prob = 0
    problem11_probabilities.append(cond_prob)

problem11_probabilities

[np.float64(0.0015307441509554297),
 np.float64(0.0015307441508450256),
 np.float64(0.0015307441432394124),
 np.float64(0.0015307438943668445),
 np.float64(0.0015307387510004125),
 np.float64(0.0015306634578246326),
 np.float64(0.0015299831809291856),
 np.float64(0.001525995482238295),
 np.float64(0.0015097453994162818),
 np.float64(0.0014613233766861324),
 np.float64(0.001352242121684195),
 np.float64(0.0011625255705667336),
 np.float64(0.0009042671762339275),
 np.float64(0.0006257545746121421),
 np.float64(0.0003838760049282565),
 np.float64(0.00021057755764174177),
 np.float64(0.00010495959267083927),
 np.float64(4.8399893298043265e-05),
 np.float64(2.0989815106644922e-05),
 np.float64(8.676066230525746e-06),
 np.float64(3.4534154491733502e-06)]

Problem 4

In [4]:
import pandas as pd

# Don't bother this
data_raw = pd.read_csv("data_2/spam.csv", encoding_errors="ignore")
data_flag = data_raw.iloc[:, 0].map({"ham": 0, "spam": 1})
data_text = data_raw.iloc[:, 1].str.lower()
sms_tuples = list(zip(data_text, data_flag))

# PROBABILITY THAT SMS CONTAINS “FREE” OR “PRIZE”
a = 0
for text, bool in sms_tuples:
    if "free" in text or "prize" in text:
        a += 1

P_B = a / len(sms_tuples)

# PROBABILITY THAT A SMS IS SPAM
b = 0
for text, bool in sms_tuples:
    if bool:
        b += 1

P_A = b / len(sms_tuples)

# PROBABILITY THAT "FREE" OR "PRIZE" IS SPAM
sms_tuples_spam = []
for text, bool in sms_tuples:
    if bool:
        sms_tuples_spam.append((text, bool))

k = 0
for text, bool in sms_tuples_spam:
    if "free" in text or "prize" in text:
        k += 1

P_BA = k / len(sms_tuples_spam)

# PROBABILITY THAT IT IS SPAM IF CONTAINS "FREE" OR "PRIZE"
P_AB = (P_BA * P_A) / P_B
print(P_AB)


'''
# ALTERNATIV LÖSNING (KORT)

import pandas as pd

# Load data
data_raw = pd.read_csv("data_2/spam.csv", encoding_errors="ignore")
data_flag = data_raw.iloc[:, 0].map({"ham": 0, "spam": 1})  # 0 = ham, 1 = spam
data_text = data_raw.iloc[:, 1].str.lower()  # Convert to lowercase
sms_tuples = list(zip(data_text, data_flag))

# Filter SMS messages containing "free" or "prize"
filtered_sms = [(text, label) for text, label in sms_tuples if "free" in text or "prize" in text]

# Extract labels (1 = spam, 0 = ham) for filtered SMS
spam_labels = [label for text, label in filtered_sms]

# Calculate P(Y=1 | "free" or "prize")
P_AB = sum(spam_labels) / len(spam_labels)
print(f"P(Y=1 | 'free' or 'prize' in SMS): {P_AB}")
'''


0.808695652173913
P(Y=1 | 'free' or 'prize' in SMS): 0.808695652173913


In [53]:
import numpy as np


def hoeffdings_range_based(X, alpha):
    log_term = np.log(2 / (1 - alpha))
    range_term = (np.max(X) - np.min(X)) ** 2
    denominator = 2 * len(X)

    epsilon = np.sqrt((log_term * range_term) / denominator)
    sample_mean = np.mean(X)

    return sample_mean - epsilon, sample_mean + epsilon


gg = []
for text, bool in sms_tuples:
    if "free" in text or "prize" in text:
        gg.append((text, bool))

spam_list = []
for text, bool in gg:
    spam_list.append(bool)

print(hoeffdings_range_based(spam_list, 0.9))



(0.7428045224509738, 0.8745867818968522)


In [54]:
# Load data
data_raw = pd.read_csv("data_2/spam.csv", encoding_errors="ignore")
data_flag = data_raw.iloc[:, 0].map({"ham": 0, "spam": 1})  # 0 = ham, 1 = spam
data_text = data_raw.iloc[:, 1].str.lower()  # Convert to lowercase
sms_tuples = list(zip(data_text, data_flag))

# Filter SMS messages containing "free" or "prize"
filtr_sms = []
for text, bool in sms_tuples:
    if text.count("free") == 2:
        filtr_sms.append((text, bool))

spm_labels = []
for text, bool in filtr_sms:
    spm_labels.append(bool)

# Calculate P(Y=1 | "free" or "prize")
P_AB = sum(spm_labels) / len(spm_labels)
print(f"P(Y=1 | 'free' or 'prize' in SMS): {P_AB}")

ff = []
for text, bool in sms_tuples:
    if text.count("free") == 2:
        ff.append((text, bool))

schozz = []
for text, bool in ff:
    schozz.append(bool)

print(hoeffdings_range_based(schozz, 0.9))

P(Y=1 | 'free' or 'prize' in SMS): 0.9736842105263158
(0.7751457258056348, 1.172222695246997)


**Problem 5**

In [55]:
import pandas as pd
import numpy as np

data = pd.read_csv("data_2/flights.csv")
unique_cities = np.unique(data[["from", "to"]])
unique_userCodes = np.unique(data["userCode"])

number_of_cities = len(unique_cities)
number_of_userCodes = len(unique_userCodes)
number_of_observations = 271887


In [56]:
def makeFreqDict(myDataList):
    '''Make a frequency mapping out of a list of data.
    Param myDataList, a list of data.
    Return a dictionary mapping each unique data value to its frequency count.
    '''
    freqDict = {}  # start with an empty dictionary
    for res in myDataList:
        if res in freqDict:  # the data value already exists as a key
            freqDict[res] = freqDict[res] + 1  # add 1 to the count using sage integers
        else:  # the data value does not exist as a key value
            freqDict[res] = 1  # add a new key-value pair for this new data value, frequency 1
    return (freqDict)  # return the dictionary created

In [57]:
# cities = XXX
unique_cities = sorted(unique_cities) # The unique cities
n_cities = len(unique_cities) # The number of unique citites

# Count the different transitions
transitions = zip(data["from"], data["to"]) # A list containing tuples ex: ('Aracaju (SE)','Rio de Janeiro (RJ)') of all transitions in the text

transition_counts = makeFreqDict(transitions) # A dictionary that counts the number of each ransition
# ex: ('Aracaju (SE)','Rio de Janeiro (RJ)'):4

indexToCity = {i: city for i, city in enumerate(unique_cities)} # A dictionary that maps the n-1 number to the n:th unique_city,
# ex: 0:'Aracaju (SE)'
cityToIndex = {city: i for i, city in enumerate(unique_cities)} # The inverse,
# ex: 'Aracaju (SE)':0

# Part 3, finding the maximum likelihood estimate of the transition matrix

transition_matrix = np.zeros((n_cities, n_cities)) # a numpy array of size (n_cities,n_cities)

for (city_from, city_to), freq in transition_counts.items():
    transition_matrix[cityToIndex[city_from], cityToIndex[city_to]] = freq


transition_matrix = transition_matrix / transition_matrix.sum(axis=1, keepdims=True)

# The transition matrix should be ordered in such a way that
# p_{'Aracaju (SE)','Rio de Janeiro (RJ)'} = transition_matrix[cityToIndex['Aracaju (SE)'],cityToIndex['Rio de Janeiro (RJ)']]
# and represents the probability of travelling Aracaju (SE)->Rio de Janeiro (RJ)


In [58]:
def stat_distr(p):
    A = p.T - np.eye(p.shape[0])
    b = np.zeros(p.shape[0])

    A[-1] = 1
    b[-1] = 1

    return np.linalg.solve(A, b)

print(stat_distr(transition_matrix))


[0.13690932 0.1132047  0.12780262 0.21081107 0.08752133 0.11210498
 0.06184532 0.06290826 0.0868924 ]


In [59]:
start_pos = [1, 0, 0, 0, 0, 0, 0, 0, 0]

three_steps = np.linalg.matrix_power(transition_matrix, 3)
final_pos = start_pos @ three_steps

print(final_pos[0])

0.13331717737273133


**Problem 6**

In [99]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

X = data_text
Y = data_flag

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42, test_size=0.2)

pipeline = Pipeline([("Vectorizer", CountVectorizer()), ("Classifier", LogisticRegression(solver="liblinear"))])
pipeline.fit(X_train, Y_train)

pred = pipeline.predict(X_test)

report = classification_report(pred, Y_test)

print(report)

accuracy = accuracy_score(pred, Y_test)
precision = precision_score(pred, Y_test)
recall = recall_score(pred, Y_test)

TP = np.sum((pred == 1) & (Y_test == 1))
FP = np.sum((pred == 1) & (Y_test == 0))
FN = np.sum((pred == 0) & (Y_test == 1))
TP = np.sum((pred == 0) & (Y_test == 0))

def DKW(delta, n):
    return np.sqrt(np.log(2 / (1 - delta)) / (2 * n))

CI_accuracy = accuracy - DKW(0.9, len(Y_test)), accuracy + DKW(0.9, len(Y_test))
print('accuracy:', CI_accuracy)

CI_recall = recall - DKW(0.9, TP+FN), recall + DKW(0.9, TP+FN)
print('recall:', CI_recall)

CI_precision = precision - DKW(0.9, TP+FP), precision + DKW(0.9, TP+FP)
print('precision:',CI_precision)

## Note that since the metrics are close to 1 already, the CI upper bound is over 1.
## You would maybe have to truncate the upper bound to 1. 

              precision    recall  f1-score   support

           0       1.00      0.98      0.99       988
           1       0.84      0.99      0.91       127

    accuracy                           0.98      1115
   macro avg       0.92      0.98      0.95      1115
weighted avg       0.98      0.98      0.98      1115

accuracy: (0.9409263580557989, 1.014230592616847)
recall: (0.9531893839523572, 1.03106258455158)
precision: (0.8006021207696175, 0.8793978792303825)
